In [ ]:
import re
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Split the raw text into lines
def preprocess_table(text):
  lines = text.strip().split('\n')

  # Initialize a list to store the processed lines
  processed_lines = []
  related_docs = ""
  # Loop through each line in the raw text
  for line in lines[2:]:  # Skip the header lines
      # Split the line into columns
      columns = line.split('|')
      if len(columns) > 3:
          document_name = columns[1].strip()
          quantity = columns[3].strip()
          related_doc = columns[2].strip()
          # Combine the document name and quantity
          processed_line = f"{quantity} {document_name} ({related_doc})"
          # Add the processed line to the list
          processed_lines.append(processed_line)

  # Join the processed lines into a single string
  result = '\n'.join(processed_lines)

  return result

In [ ]:
def preprocess_doc(text):
    table_header_pattern = r"\|Tên giấy tờ\|Mẫu đơn, tờ khai\|Số lượng\|((?:[^|\n]*\|?)+)"

    # Find the table header and extract all subsequent lines until a non-table line is encountered
    processed_text = text
    match = re.search(table_header_pattern, processed_text)
    while match:
        table_header = match.group(1).strip()
        table_content = "|Tên giấy tờ|Mẫu đơn, tờ khai|Số lượng|\n"
        remaining_text = processed_text[match.end():].strip()
        lines = remaining_text.splitlines()

        for line in lines:
            if line.startswith("|"):
                table_content += line.strip() + "\n"
            else:
                table_content = table_content[:-1]
                break
        new_table_content = preprocess_table(table_content)
        table_start = processed_text.find(table_content)  # Find the start position of the matched table
        processed_text = processed_text[:table_start] + new_table_content + processed_text[table_start + len(table_content):]
        match = re.search(table_header_pattern, processed_text)
    return processed_text


In [ ]:
def process_files_in_folder(folder_path):
    # Iterate over all files in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.endswith('.txt'):  # Ensure it's a file and ends with .txt
            # Read the current file content
            with open(file_path, 'r', encoding='utf-8') as file:
                file_content = file.read()

            # Apply the function ggez to modify the text
            modified_content = preprocess_doc(file_content)

            # Save the modified content back to the file
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(modified_content)

In [ ]:
process_files_in_folder("./drive/MyDrive/txt_file_old")

KeyboardInterrupt: 